In [ ]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
from utils.configs import PRE_SALE_BASE_URL, PRE_SALE_URLS_FRAGMENTS, PRE_SALE_COLUMN_NAME
from utils.helper_func import build_complete_urls , combined_df, parse_admin_region, to_year_quarter

In [ ]:
urls = build_complete_urls(PRE_SALE_BASE_URL, PRE_SALE_URLS_FRAGMENTS)
print(urls)

In [ ]:
df = combined_df(urls, "20250711")

In [ ]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\pre_sale_data\raw_data"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "預售屋買賣主檔_11006_11406.csv"
out_path = os.path.join(output_dir,  csv_fn)
df.to_csv(out_path, index=False, encoding='utf-8-sig')

In [ ]:
input_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\pre_sale_data\raw_data"
csv_fn = "預售屋買賣主檔_11006_11406.csv"
input_path = os.path.join(input_dir,  csv_fn)
presale_df = pd.read_csv(input_path)

In [ ]:
presale_df

In [ ]:
presale_df = presale_df.rename(columns= PRE_SALE_COLUMN_NAME, inplace=False)
proc_df = presale_df.copy()
proc_df = proc_df[["備查編號", "縣市", "坐落街道", "建物型態", "社區名稱", "棟號", "交易日期", "總面積", "交易總價",  
"建物單價", "樓層", "解約情形",   "主要用途",  "車位總價", "車位筆數",  "備註", "經度", "緯度"]]

In [ ]:
# 在 proc_df 裡面新增一個「行政區」欄位
district_series = proc_df["坐落街道"].apply(parse_admin_region)
idx = proc_df.columns.get_loc("縣市") + 1
proc_df.insert(loc=idx,
                column="行政區",
                value=district_series)

# 新增「交易年月」欄位
proc_df['交易年月'] = proc_df['交易日期'].str.replace(r'(\d+)/(\d+)/\d+', r'\1\2', regex=True)

proc_df['交易總價'] = pd.to_numeric(proc_df['交易總價'].str.replace(',', ''), errors='coerce')
proc_df['交易總價'] = (proc_df['交易總價'] / 10000).round(0).astype(int)

proc_df['建物單價'] = pd.to_numeric(proc_df['建物單價'].str.replace(',', ''), errors='coerce')
proc_df['建物單價'] = (proc_df['建物單價'] / 10000).round(1).fillna(0).astype(float)

# 1. 先準備好要插入的 Series
sq_series = proc_df["交易年月"].apply(to_year_quarter)

# 2. 找到「銷售起始時間」的欄位位置，並 +1 得到插入索引
idx = proc_df.columns.get_loc("交易日期") + 1

# 3. 用 insert() 把新欄位放進去
proc_df.insert(loc=idx,
                column="交易年季",
                value=sq_series)

In [ ]:
proc_df = proc_df[proc_df['備查編號'].notna()]
# 重置索引，並捨棄原本的索引欄位
proc_df = proc_df.reset_index(drop=True)
proc_df

In [ ]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\pre_sale_data\processed"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "預售屋買賣主檔_11006_11406.csv"
out_path = os.path.join(output_dir,  csv_fn)
proc_df.to_csv(out_path, index=False, encoding='utf-8-sig')